In [2]:
from domino.training_sets import TrainingSetClient, model #interface to define and initialize a training data set
import pandas as pd

df=pd.read_csv("train.csv")
training_set_name="dbs-test-data"

tsv=TrainingSetClient.create_training_set_version(
  training_set_name=training_set_name,
  df=df,
  key_columns=[],
  target_columns=["price"], #the value of this property would be used later in the DataCaptureClient’s signature
  exclude_columns=[],
  monitoring_meta=model.MonitoringMeta(
    timestamp_columns=[],
    categorical_columns=[],
    ordinal_columns=[],
    ),
  meta={"year": "2022"}
  )

print(f"Training Set Version {tsv.training_set_name}:{tsv.number}") #properties to invoke on the TrainingSetClient object to get it’s name and version number
df.head()

Training Set Version dbs-test-data:1


,area,price
0,2600,550000
1,3000,565000
2,3200,610000
3,3600,680000
4,4000,725000


In [3]:
import numpy
from sklearn import linear_model
train=linear_model.LinearRegression() #y=mx+c
train.fit(df[['area']].values,df.price) #train model on our dataset
train.coef_ #slope

array([135.78767123])

In [4]:
train.intercept_ #point of inception

180616.43835616432

In [5]:
train.predict([[5000]]) #invoking the model on data point 5000

array([859554.79452055])

In [6]:
import pickle

# Saving to disk 
with open('serialized_model', 'wb') as files:
  pickle.dump(train, files)

In [7]:
# for how the file and the method you'd pass into the model api would look like

import uuid 
import datetime

#domino module to capture and store the prediction dat 
from domino_data_capture.data_capture_client import DataCaptureClient

features=["area"]
target=["price"]   #value of this arg/var is what you had set prior when defining the training data set
data_capture_client=DataCaptureClient(features,target)   #DataCaptureClient's signature

model_file="serialized_model"   #loading the same model saved to disk
model=pickle.load(open(model_file,'rb'))

def callable_function (area):
  feature=[[area]]
  price_predict=model.predict(feature)
  event_id=uuid.uuid4()
  timestamp=datetime.datetime.now(datetime.timezone.utc).isoformat()

#invoking the capturePrediction method on the DataCaptureClient object 
  data_capture_client.capturePrediction(feature,price_predict,event_id=event_id,timestamp=timestamp)

In [8]:
callable_function(5000) #invoking the model api exposed endpoint

The information shown below represents the structure and format of the prediction data recorded by Domino when this model is deployed as a Domino Model API

{"predictions": {"price": 859554.7945205481}, "features": {"area": [5000]}, "metadata": null, "timestamp": "2025-01-22T09:54:17.333325+00:00", "__domino_timestamp": "2025-01-22T09:54:17.333352+00:00", "event_id": "cf1a9848-59ff-449a-99fe-c2f9670e2001", "prediction_probability": null, "sample_weight": null, "instance_id": null}


In [9]:
def callable_function1 (area):
  feature=[[area]]
  price_predict=model.predict(feature)
  price_predict=price_predict.tolist()
  return_val=json.dumps(price_predict)
  event_id=uuid.uuid4()
  timestamp=datetime.datetime.now(datetime.timezone.utc).isoformat()
  pred_capture=data_capture_client.capturePrediction(feature,price_predict,event_id=event_id,timestamp=timestamp) #invoking the capturePrediction method on the DataCaptureClient object
  print(f"Output from data_capture_client : {pred_capture}")
  return return_val